# Section 7

### Today
<ul>
<li>Fully Convolutional Networks</li>
<li><a href='https://www.tensorflow.org/api_docs/python/tf/image'>Up-Convolutions</a></li>
<li><a href='https://www.tensorflow.org/versions/master/api_docs/python/tf/data/TFRecordDataset'>TensorFlow Datasets</a></li>
<li><a href='https://www.tensorflow.org/api_docs/python/tf/layers/conv2d'>Skip Connections</a></li>
</ul>

### Related Reading
<ul>
<li><a href='https://distill.pub/2016/deconv-checkerboard/'>Deconvolution Vizualation</a></li>
<li><a href='https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf'>Fully Convolutional Networks for Semantic Segmentation</a></li>
<li><a href='https://arxiv.org/pdf/1603.07285v1.pdf'>A guide to convolution arithmetic for deep
learning</a></li>
<li><a href='http://www.matthewzeiler.com/wp-content/uploads/2017/07/cvpr2010.pdf'>Deconvolutional Networks</a></li>
<li><a href='https://www.tensorflow.org/programmers_guide/datasets'>TensorFlow Programmers Guide on Importing Data</a> - focuses on newer Dataset API tf.contrib.data.Dataset (not what we are using for now but has a similar API).</li>
</ul>

In [ ]:
import tensorflow as tf
import numpy as np
import util

## Datasets - <a href='https://www.tensorflow.org/versions/master/api_docs/python/tf/data/TFRecordDataset'>tf.data.TFRecordDataset</a>
"The TFRecord file format is a simple record-oriented binary format that many TensorFlow applications use for training data. The tf.contrib.data.TFRecordDataset class enables you to stream over the contents of one or more TFRecord files as part of an input pipeline."

In [ ]:
def parser(record):
    # Parse the TF record
    parsed = tf.parse_single_example(record, features={
       'height': tf.FixedLenFeature([], tf.int64),
       'width': tf.FixedLenFeature([], tf.int64),
       'image_raw': tf.FixedLenFeature([], tf.string),
       'label_raw': tf.FixedLenFeature([], tf.string)
    })
    # Load the data and format it
    H = tf.cast(parsed['height'], tf.int32)
    W = tf.cast(parsed['width'], tf.int32)
    image = tf.reshape(tf.decode_raw(parsed["image_raw"], tf.uint8), [H,W,3])
    label = tf.reshape(tf.decode_raw(parsed["label_raw"], tf.uint8), [H,W])
    # Perform additional preprocessing on the parsed data.
    return image, label

def data_augmentation(image, label):
    # TODO: Apply some data augmentation, namely cropping and mirroring for faster training
    return image, label

def load_dataset(tfrecord, augment=True):
    # Load the dataset
    dataset = tf.contrib.data.TFRecordDataset(tfrecord)

    # Parse the tf record entries
    dataset = dataset.map(parser)
    if augment:
        dataset = dataset.map(data_augmentation)

    # Shuffle the data, batch it and run this for multiple epochs
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(32)
    dataset = dataset.repeat()
    return dataset

# Lets clear the tensorflow graph, so that you don't have to restart the notebook every time you change the network
tf.reset_default_graph()

train_data = load_dataset('train.tfrecord')
valid_data = load_dataset('valid.tfrecord')

# Create an iterator for the datasets
# The iterator allows us to quickly switch between training and validataion
iterator = tf.contrib.data.Iterator.from_structure(train_data.output_types,train_data.output_shapes)

# and fetch the next images from the dataset (every time next_image is evaluated a new image set of 32 images is returned)
next_image, next_label = iterator.get_next()

# Define operations that switch between train and valid
switch_train_op = iterator.make_initializer(train_data)
switch_valid_op = iterator.make_initializer(valid_data)


## Fully Convolutional Networks (FCNs)
<b>Key Idea</b>: Whole Image Inputs -- Whole Image Ground Truths

## Skip Connections
"Combining fine layers and coarse layers lets the model make local predictions that respect global structure." - <a href='https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf'>Fully Convolutional Networks for Semantic Segmentation</a>


In [ ]:
# Option 1: Use tensors of same shape and add


In [ ]:
# Option 2: Use tensors of different shapes and add
